


<font size='10' color = 'E3A440'>**Mégadonnées et techniques avancées démystifiées**</font>
=======
<font color = 'E3A440'>*Nouvelles méthodes d’analyse et leur implication quant à la gestion des mégadonnées en SSH (partie 1)*</font>
=============


Cet atelier s’inscrit dans le cadre de la formation [Mégadonnées et techniques avancées démystifiées](https://www.4point0.ca/2022/08/22/formation-megadonnees-demystifiees/) (séance 6).

Les sciences humaines et sociales sont souvent confrontées à l’analyse de données non structurées, comme le texte. Après avoir préparé les données, plusieurs techniques d’analyse venant de l’apprentissage automatique peuvent être utilisées. Pendant cet atelier, les participants seront initiés aux méthodes supervisées et non supervisées à des buts d’analyse avec Python.

Note : Cet atelier se poursuit lors d’une 2e séance le 10 novembre.

Structure de l'atelier :
1. Presentation of sections 1 and 2 in a plenary mode (20 minutes)
2. Individual work on section 3 (20 minutes)
3. Group work on section 4 (60 minutes)
4. Plenary session with groups presentations (20 minutes)

Ce tutoriel ne peut pas être consideré exaustif .... 

### Authors: 
- Bruno Agard <bruno.agard@polymtl.ca>
- Davide Pulizzotto <davide.pulizzotto@polymtl.ca>

### Table of Contents
Bruno Agard

Département de Mathématiques et de génie Industriel

École Polytechnique de Montréal

# Préparation environnement

In [2]:
# Downloading of data from the GitHub project
!rm -rf Donnees_demystifiees_seance_6/
!git clone https://github.com/puli83/Donnees_demystifiees_seance_6

Cloning into 'Donnees_demystifiees_seance_6'...
remote: Enumerating objects: 55, done.
remote: Counting objects: 100% (55/55), done.
remote: Compressing objects: 100% (52/52), done.
remote: Total 55 (delta 16), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (55/55), done.


In [6]:
# Import modules
import nltk
import pandas as pd
import numpy as np
import os
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('averaged_perceptron_tagger')
nltk.download('universal_tagset')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package universal_tagset to /root/nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!


True

# <font color = 'E3A440'>*Préparation des données textulles*</font>

L'analyse de données textuelles implique la transformation d'un texte en un objet mathematique qui peut être utilisé par des algorithmes et des modèles statistique. Cette étape est importante car permet de **structurer** des données non structurées, comme le texte.


###  <font color = 'E3A440'>**Étapes fondamentales du prétraitement**</font>

Prenons une phrase pour decortiquer les passages que nous pemrettent de la transformer en information structurée.

In [3]:
sentence = """At eight o'clock, on Thursday morning, the great Arthur didn't feel VERY good."""

In [4]:
len(sentence)

78

#### <font color = 'E3A440'>*1. Tokenisation*</font>

Cette étape consiste à couper la phrase en unités linguistiques élémantaire et dotées de sens, ce qui est gnralement appelé le "mot".

Dans le module `nltk`, il existe une fonction qui permet cette opération, soit `word_tokenize()`.

In [5]:
# La function word_tokenize() prend la phrase comme argument.
words = nltk.word_tokenize(sentence)
print(words)
len(words)

['At', 'eight', "o'clock", ',', 'on', 'Thursday', 'morning', ',', 'the', 'great', 'Arthur', 'did', "n't", 'feel', 'VERY', 'good', '.']


17

#### <font color = 'E3A440'>*2. Analyse morphosyntaxique*</font>

Après avoir identifé tous les mots, il est possible de analyser leur rôle morphosyntaxique, à des fins d'analyse et/ou filtrage. 

In [6]:
 # La function word_tokenize() prend la liste de mots comme argument.
words_pos = nltk.pos_tag(words, tagset='universal')
print(words_pos)
len(words_pos)

[('At', 'ADP'), ('eight', 'NUM'), ("o'clock", 'NOUN'), (',', '.'), ('on', 'ADP'), ('Thursday', 'NOUN'), ('morning', 'NOUN'), (',', '.'), ('the', 'DET'), ('great', 'ADJ'), ('Arthur', 'NOUN'), ('did', 'VERB'), ("n't", 'ADV'), ('feel', 'VERB'), ('VERY', 'ADV'), ('good', 'ADJ'), ('.', '.')]


17

Voici la liste de possible POS tags:

| **POS** | **DESCRIPTION**           | **EXAMPLES**                                      |
| ------- | ------------------------- | ------------------------------------------------- |
| ADJ     | adjective                 | big, old, green, incomprehensible, first      |
| ADP     | adposition                | in, to, during                                |
| ADV     | adverb                    | very, tomorrow, down, where, there            |
| AUX     | auxiliary                 | is, has (done), will (do), should (do)        |
| CONJ    | conjunction               | and, or, but                                  |
| CCONJ   | coordinating conjunction  | and, or, but                                  |
| DET     | determiner                | a, an, the                                    |
| INTJ    | interjection              | psst, ouch, bravo, hello                      |
| NOUN    | noun                      | girl, cat, tree, air, beauty                  |
| NUM     | numeral                   | 1, 2017, one, seventy-seven, IV, MMXIV        |
| PART    | particle                  | ’s, not                                      |
| PRON    | pronoun                   | I, you, he, she, myself, themselves, somebody |
| PROPN   | proper noun               | Mary, John, London, NATO, HBO                 |
| PUNCT   | punctuation               | ., (, ), ?                                    |
| SCONJ   | subordinating conjunction | if, while, that                               |
| SYM     | symbol                    | $, %, §, ©, +, −, ×, ÷, =, :)               |
| VERB    | verb                      | run, runs, running, eat, ate, eating          |
| X       | other                     | sfpksdpsxmsa                                  |
| SPACE   | space                     |                                                   |


#### <font color = 'E3A440'>*3. Retirer la ponctuation*</font>

Une autre opération consiste à retirer la ponctuation. Ce type de filtrage reduit le nombre de sugne graphique qui participent le moin à la constructuion de la sémantique de la prhase. 
Dans certain contexte, comme en stylometrie, ce processus est appliquée avec de terchniques plus sofistiquées. 

In [7]:
# La ligne de code suivant itére sur chaque signe graphique et retient ceux qui contiennet de caracteres alphanumérique.
words_pos = [(w, pos) for w, pos in words_pos if w.isalnum()]
print(words_pos)
len(words_pos)
# Ikl est possible aussi d'utiilser le résultat de l'analyse morphosyntaxique pour éliminer la ponctuaction
# words_pos = [(w, pos) for w, pos in words_pos if pos != '.']
# print(words_pos)

[('At', 'ADP'), ('eight', 'NUM'), ('on', 'ADP'), ('Thursday', 'NOUN'), ('morning', 'NOUN'), ('the', 'DET'), ('great', 'ADJ'), ('Arthur', 'NOUN'), ('did', 'VERB'), ('feel', 'VERB'), ('VERY', 'ADV'), ('good', 'ADJ')]


12

#### <font color = 'E3A440'>*4. Convertir chaque caractère en minuscule*</font>

Cette étape constitue une première opéraiton de normalisation des mots et leur réduction à une forme graphique unique. Ce genre d'étape permet de regrouper chaque occurence d'un mot sous une seule forme.

In [8]:
# La ligne de code suivant itére sur chaque signe graphique et le transforme en minuscule.
words_pos = [(w.lower(), pos) for w, pos in words_pos]
print(words_pos)

[('at', 'ADP'), ('eight', 'NUM'), ('on', 'ADP'), ('thursday', 'NOUN'), ('morning', 'NOUN'), ('the', 'DET'), ('great', 'ADJ'), ('arthur', 'NOUN'), ('did', 'VERB'), ('feel', 'VERB'), ('very', 'ADV'), ('good', 'ADJ')]


#### <font color = 'E3A440'>*5. retirer les stopwrods (mots vides)*</font>

Une autre opération de filtrage constitue dan l'élimination de mots fonctionnels. Cette liste de mots contient tout les connecteurs de phrase, comme "et", "mais", "toutefois" et de mots avec faible valeur sémantique, comem les verbes modaux. 
Comme d'autres opéraiton de filtrage, l'enjeuy est celui de nettoyer le plus possible le vocabulaire et de reduyire toutes les occurrences d'un mot sous une forme graphique unique.

In [9]:
# Nous importons la liste de stopword en anglais
from nltk.corpus import stopwords
print(stopwords.words("english"))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [10]:
# La ligne de code suivant itére sur chaque signe graphique et garde ce qui n sont pas dans la liste de stopword.
words_pos = [(w, pos) for w, pos in words_pos if w not in stopwords.words("english")]
print(words_pos)
len(words_pos)

[('eight', 'NUM'), ('thursday', 'NOUN'), ('morning', 'NOUN'), ('great', 'ADJ'), ('arthur', 'NOUN'), ('feel', 'VERB'), ('good', 'ADJ')]


7

#### <font color = 'E3A440'>*6. Rammener les mots à leur racine*</font> 

En suivant le même objectif, nous retirons le suffixe morphologique des mots, ce qui augmente le niveau de réduction de chaque occurrence d'un mot à une unique forme graphique.

Ils existent deux méthode fondamentales: la racinisaiton et la lemmatisation.
La première reduit les occurence à une racine qui est inférée au moyen de plusieur techniques, l'autre est la réduciton de l'occurrence à son lemme. 

In [11]:
# Racinisation: technique Porter
from nltk.stem.porter import PorterStemmer
stemmed_pos = [(PorterStemmer().stem(w), pos) for w, pos in words_pos]
print(stemmed_pos)

[('eight', 'NUM'), ('thursday', 'NOUN'), ('morn', 'NOUN'), ('great', 'ADJ'), ('arthur', 'NOUN'), ('feel', 'VERB'), ('good', 'ADJ')]


In [12]:
# Racinisation: technique Lancaster
from nltk.stem import LancasterStemmer
stemmed_pos = [(LancasterStemmer().stem(w), pos) for w, pos in words_pos]
print(stemmed_pos)

[('eight', 'NUM'), ('thursday', 'NOUN'), ('morn', 'NOUN'), ('gre', 'ADJ'), ('arth', 'NOUN'), ('feel', 'VERB'), ('good', 'ADJ')]


In [13]:
# Lemmatisaiton: utilisant le thesaurus wordnet
from nltk.stem.wordnet import WordNetLemmatizer
lemmed_pos = [(WordNetLemmatizer().lemmatize(w), pos) for w, pos in words_pos]
print(lemmed_pos)

[('eight', 'NUM'), ('thursday', 'NOUN'), ('morning', 'NOUN'), ('great', 'ADJ'), ('arthur', 'NOUN'), ('feel', 'VERB'), ('good', 'ADJ')]


#### <font color = 'E3A440'>*7. Filtrage selon le rôle morphosyntaxique*</font>

Le filtrage des unités lexicales peut s'étendre jusqu'à l'élimination d'unités qui ne font pas partie d'une liste de rôles morphosyntaxique prédéfinie. 

In [14]:
# Retenir seulement les noms et les adjectifs
lemmed_pos = [(w, pos) for w, pos in words_pos if pos in ['NOUN','ADJ']]
print(lemmed_pos)

[('thursday', 'NOUN'), ('morning', 'NOUN'), ('great', 'ADJ'), ('arthur', 'NOUN'), ('good', 'ADJ')]


## <font color = 'E3A440'>Traitement d'un texte</font>

Le prétraitement d'un corpus de recherche peut mettre en place plusieurs autres étapes. La plus importante est la **segmentation**. 

### <font color = 'E3A440'>*1. Segmentation*</font>

Tout dépendant de l'objectif de l'analyse, un segment peut être un document, un paragraphe, une concordance, un groupe de phrases, une phrase, etc.



In [15]:
text = """At eight o'clock, on Thursday morning, the great Arthur didn't feel VERY good.
          The following morning, at nine, Arthur felt better.
          A dog run in the street."""
len(text)

175

Dans le bloc de code suivant, nous faisons une segmentation par pharse.

In [34]:
sentences = nltk.sent_tokenize(text)
print(sentences)
len(sentences)

["At eight o'clock, on Thursday morning, the great Arthur didn't feel VERY good.", 'The following morning, at nine, Arthur felt better.', 'A dog run in the street.']


3

### <font color = 'E3A440'>*2. Annotation et nettoyage*</font>

Tout dépendant de l'objectif de l'analyse, un segment peut être un document, un paragraphe, une concordance, un groupe de phrases, une phrase, etc.


#### <font color = 'E3A440'>*2.1 Création d'une fonction*</font>

La cération de funciton est utile pour plusieurs raison. Dans notre casm, nous voulons englober Souvent, il est utile de créer de fonctions pour executer accomplir pluseurs étapes

In [19]:
# To run this function proprlely, you need to import modules needed
def CleaningText(text_as_string, language = 'english', reduce = '', list_pos_to_keep = [], Stopwords_to_add = []):
    from nltk.corpus import stopwords

    words = nltk.word_tokenize(text_as_string)
    words_pos = nltk.pos_tag(words, tagset='universal')
    words_pos = [(w, pos) for w, pos in words_pos if w.isalnum()]
    words_pos = [(w.lower(), pos) for w, pos in words_pos]
    
    if reduce == 'stem': 
        from nltk.stem.porter import PorterStemmer
        reduced_words_pos = [(PorterStemmer().stem(w), pos) for w, pos in words_pos]
        
    elif reduce == 'lemma':
        from nltk.stem.wordnet import WordNetLemmatizer
        reduced_words_pos = [(WordNetLemmatizer().lemmatize(w), pos) for w, pos in words_pos]
    else:
        import warnings
        reduced_words_pos = words_pos
        warnings.warn("Warning : any reduction was made on words! Please, use \"reduce\" argument to chosse between 'stem' or  'lemma'")
    if list_pos_to_keep:
        reduced_words_pos = [(w, pos) for w, pos in reduced_words_pos if pos in list_pos_to_keep]
    else:
        import warnings
        warnings.warn("Warning : any POS filtering was made. Pleae, use \"list_pos_to_keep\" to create a list of POS tag to keep.")
    
    list_stopwords = stopwords.words(language) + Stopwords_to_add
    reduced_words_pos = [(w, pos) for w, pos in reduced_words_pos if w not in list_stopwords and len(w) > 1 ]
    return reduced_words_pos



In [145]:
words = nltk.word_tokenize(text_as_string)
words_pos = nltk.pos_tag(words, tagset='universal')
words_pos = [(w, pos) for w, pos in words_pos if w.isalnum()]
words_pos = [(w.lower(), pos) for w, pos in words_pos]
from nltk.stem.wordnet import WordNetLemmatizer
reduced_words_pos = [(WordNetLemmatizer().lemmatize(w), pos) for w, pos in words_pos]
reduced_words_pos = [(w, pos) for w, pos in reduced_words_pos if pos in ['NOUN','ADJ','VERB']]
list_stopwords = stopwords.words(language) + ['http']
words_pos = [(w, pos) for w, pos in reduced_words_pos if w not in list_stopwords ]
reduced_words_pos



[('entire', 'ADJ'),
 ('swiss', 'ADJ'),
 ('football', 'NOUN'),
 ('league', 'NOUN'),
 ('is', 'VERB'),
 ('hold', 'NOUN'),
 ('postponing', 'VERB'),
 ('game', 'NOUN'),
 ('professional', 'ADJ'),
 ('amateur', 'ADJ'),
 ('level', 'NOUN'),
 ('coronavirus', 'ADJ'),
 ('http', 'NOUN')]

#### <font color = 'E3A440'>*2.2 Application function nettoyage*</font>

Maintenant, nous pouvons apliquer cette function à chacun de not segment. dans notre cas il s'agit de phrases.

In [37]:
cleaned_sentences = [CleaningText(sent) for sent in sentences]
print(cleaned_sentences)

[[('eight', 'NUM'), ('thursday', 'NOUN'), ('morning', 'NOUN'), ('great', 'ADJ'), ('arthur', 'NOUN'), ('feel', 'VERB'), ('good', 'ADJ')], [('following', 'ADJ'), ('morning', 'NOUN'), ('nine', 'NUM'), ('arthur', 'NOUN'), ('felt', 'VERB'), ('better', 'ADV')], [('dog', 'NOUN'), ('run', 'NOUN'), ('street', 'NOUN')]]


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: UserWarning: Warning : any reduction was made on words! Please, use "reduce" argument to chosse between 'stem' or  'lemma'
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:24: UserWarning: Warning : any POS filtering was made. Pleae, use "list_pos_to_keep" to create a list of POS tag to keep.


In [ ]:
cleaned_sentences = [CleaningText(sent, reduce = 'lemma', list_pos_to_keep = ['']) for sent in sentences]
print(cleaned_sentences)

#### <font color = 'E3A440'>*2.3 Fréquence mots*</font>

Quelle est la fréquence des mots de notre texte? Commencon par retirer le POS tag, pour obtenir une liste de listes de mots ( et non une liste de tuple mot-pos).

In [48]:
freqs_in_text = nltk.FreqDist([w for sent in cleaned_sentences for w, pos in sent ])
freqs_in_text

FreqDist({'morning': 2, 'arthur': 2, 'eight': 1, 'thursday': 1, 'great': 1, 'feel': 1, 'good': 1, 'following': 1, 'nine': 1, 'felt': 1, ...})

### <font color = 'E3A440'>*3. Vectorisation*</font>

Généralement, pour utiliser le texte dans un contexte d'analyse de données ou d'apprentissage automatique, ce texte doit être transformé dans un objet mathématique approprié. 
Le modèle le plus simple et connu est le "bags-of-words", dans lequel chaque document (ou chaque mot) est défini par un certain nombre d'unités lexicales qui le caractérise. 


The main step in text mining is to convert unstructured textual data into a mathematical model to be used in statistical learning. Thus, we need to create a <font color='E3A440'>**Document-Term matrix**</font>, a matrix $n \times w$, where $n$ is the number of text segments and $w$ is the number of textual features selected.The textual feature can have different nature. In the simplest model, these features correspond to the set of types that resume each token of the corpus. In other terms, $w$ is the number of features that characterize a segment of text. The matrix is generally represented as follows:  
 
$$X = \begin{bmatrix} 
x_{11} & x_{12} & \ldots & x_{1w} \\
\vdots & \vdots       &  \ddots      & \vdots \\ 
x_{n1} & x_{12} & \ldots & x_{nw} \\
\end{bmatrix}
$$ 
 
\\

In [38]:
# Initialisation de l'objet
from nltk.corpus import stopwords

def identity_tokenizer(text):
    return text

# Transforming the word in frequencies
vectorized = CountVectorizer(lowercase = False, # Convert all characters to lowercase before tokenizing
                             min_df = 1, # Ignore terms that have a document frequency strictly lower than the given threshold 
                             max_df = 10, # Ignore terms that have a document frequency strictly higher than the given threshold (corpus-specific stop words)
                             stop_words = stopwords.words('english'), # Remove the list of words provided
                             ngram_range = (1, 1), # Get the lower and upper boundary of the range of n-values for different word n-grams or char n-grams to be extracted
                             tokenizer=identity_tokenizer) # Override the string tokenization step while preserving the preprocessing and n-grams generation steps

Utilisation du "vectorizer" avec une liste de listes de mot (et non une liste de tuple de mots-pos).

In [49]:
# Liste de liste de mots:
[[w for w, pos in sent] for sent in cleaned_sentences]

[['eight', 'thursday', 'morning', 'great', 'arthur', 'feel', 'good'],
 ['following', 'morning', 'nine', 'arthur', 'felt', 'better'],
 ['dog', 'run', 'street']]

In [79]:
# pplication du vectorizer
freq_term_DTM = vectorized.fit_transform([[w for w, pos in sent] for sent in cleaned_sentences])
print(pd.DataFrame(freq_term_DTM.todense(), columns =  [k for k, v in sorted(vectorized.vocabulary_.items(), key=lambda item: item[1])] ))

   arthur  better  dog  eight  feel  felt  following  good  great  morning  \
0       1       0    0      1     1     0          0     1      1        1   
1       1       1    0      0     0     1          1     0      0        1   
2       0       0    1      0     0     0          0     0      0        0   

   nine  run  street  thursday  
0     0    0       0         1  
1     1    0       0         0  
2     0    1       1         0  


## Exercise : Sentiment analysis on a COVID-19 twetter dataset

In [135]:
ROOT_DIR='Donnees_demystifiees_seance_6/'
DATA_DIR=os.path.join(ROOT_DIR, 'Data')
import zipfile
from datetime import datetime

#Unzips the dataset and gets the TSV dataset
#with zipfile.ZipFile(os.path.join(DATA_DIR,'4POINT0_Top_50_tweet_profiles.zip'), 'r') as zip_ref:
#    zip_ref.extractall(DATA_DIR)

df = pd.read_pickle(os.path.join(DATA_DIR,'Top_50_tweet_profiles.pkl')).sample(5000).reset_index()

In [70]:
df.shape

(5000, 14)

In [117]:
df.dtypes

index                                  int64
Tweet Id                              object
Tweet URL                             object
Tweet Posted Time             datetime64[ns]
Tweet Content                         object
Tweet Type                            object
Client                                object
Retweets received                      int64
Likes received                         int64
User Id                               object
Name                                  object
Username                              object
Verified or Non-Verified              object
Profile URL                           object
Protected or Not Protected            object
neg                                  float64
neu                                  float64
pos                                  float64
compound                             float64
compound_category                   category
dtype: object

In [ ]:
df.iloc[0]

In [ ]:
df['Tweet Content']

Cette opération prendra 2 minutes, donc, nous vous suggérons de continuer la lecture dce l'exercices pour vous faire une idée de ce qui s'en vient.

In [149]:
cleaned_tweets = [CleaningText(sent, reduce = 'lemma', list_pos_to_keep = ['ADJ'], Stopwords_to_add=['http']) for sent in list(df['Tweet Content'])]

In [150]:
# Initialisation de l'objet
from nltk.corpus import stopwords

def identity_tokenizer(text):
    return text

# Transforming the word in frequencies
vectorized = CountVectorizer(lowercase = False, # Convert all characters to lowercase before tokenizing
                             min_df = 10, # Ignore terms that have a document frequency strictly lower than the given threshold 
                             max_df = 4500, # Ignore terms that have a document frequency strictly higher than the given threshold (corpus-specific stop words)
                             stop_words = stopwords.words('english'), # Remove the list of words provided
                             ngram_range = (1, 1), # Get the lower and upper boundary of the range of n-values for different word n-grams or char n-grams to be extracted
                             tokenizer=identity_tokenizer) # Override the string tokenization step while preserving the preprocessing and n-grams generation steps

In [151]:
freq_term_DTM = vectorized.fit_transform([[w for w, pos in sent] for sent in cleaned_tweets])
freq_term_DTM

/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:401: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ["'", 'b', 'c', 'e', 'f', 'g', 'h', 'j', 'l', 'n', 'p', 'r', 'u', 'v', 'w'] not in stop_words.
  % sorted(inconsistent)


<5000x71 sparse matrix of type '<class 'numpy.int64'>'
	with 2067 stored elements in Compressed Sparse Row format>

In [80]:
from nltk.sentiment import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')
sia = SentimentIntensityAnalyzer()

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [64]:
print(sia.polarity_scores("Wow, NLTK is really powerful!"))
sia.polarity_scores("NLTK is not bad!")

{'neg': 0.0, 'neu': 0.295, 'pos': 0.705, 'compound': 0.8012}


{'neg': 0.0, 'neu': 0.488, 'pos': 0.512, 'compound': 0.484}

In [65]:
sia.polarity_scores("NLTK is bad!")

{'neg': 0.655, 'neu': 0.345, 'pos': 0.0, 'compound': -0.5848}

In [139]:
datasent = df.apply(lambda x: sia.polarity_scores(x['Tweet Content']), 1)
df = df.join(pd.DataFrame(list(datasent)))

In [141]:
df['compound']

0       0.0000
1       0.0000
2       0.4019
3      -0.0772
4       0.9632
         ...  
4995    0.6124
4996    0.8360
4997    0.0000
4998    0.4466
4999    0.0000
Name: compound, Length: 5000, dtype: float64

In [140]:
df['compound'].describe()

count    5000.000000
mean        0.217127
std         0.419101
min        -0.982500
25%         0.000000
50%         0.000000
75%         0.585900
max         0.993700
Name: compound, dtype: float64

In [142]:
bins = [-np.inf, -0.5, 0.5, 1]
names = ['negative', 'neu', 'positive']
df['compound_category']  = pd.cut(df['compound'], bins, labels=names, include_lowest =True)

In [ ]:
df[df['compound_category'] == 'positive']

In [105]:
from collections import Counter
Counter(df['compound_category'])

Counter({'neu': 3318, 'positive': 1374, 'negative': 308})

In [143]:
logical_vector = df['compound_category'] == 'positive'

In [92]:
sum(logical_vector)

286

In [110]:
def GetLexicalSpecificities(freq_term_DTM, logical_vector):
    # This code ref takes inspiration from this python module : https://pypi.org/project/corpus-toolkit/
    # and its main script:  https://github.com/kristopherkyle/corpus_toolkit/blob/master/corpus_toolkit/corpus_tools.py
    # which is based on this paper: https://aclanthology.org/J93-1003/
    import math
    df_freq_target = pd.DataFrame(np.asarray(freq_term_DTM[logical_vector].sum(0).T).reshape(-1))
    df_freq_target.index = [word for (word,idx) in sorted(vectorized.vocabulary_.items(), key= lambda x:x[1])]
    df_freq_target.columns = ['freq1']
    df_freq_target['freq2'] = np.asarray(freq_term_DTM[~(logical_vector)].sum(0).T).reshape(-1)
    df_freq_target['tot'] = df_freq_target['freq1'] + df_freq_target['freq2']

    df_freq_target['freq1'] = df_freq_target['freq1'].apply(lambda x: 0.0000001 if x == 0 else x).astype(float)
    df_freq_target['freq2'] = df_freq_target['freq2'].apply(lambda x: 0.0000001 if x == 0 else x).astype(float)
    #
    df_freq_target['freq1_norm'] = df_freq_target['freq1']/df_freq_target['freq1'].sum() * 1000000
    df_freq_target['freq2_norm'] = df_freq_target['freq2']/df_freq_target['freq2'].sum() * 1000000
    #
    df_freq_target['fraction'] = df_freq_target['freq1_norm'] / df_freq_target['freq2_norm']
    df_freq_target['Log-likelihood Ratio'] = df_freq_target['fraction'].apply(math.log2)
    frequency_threshold = 10 # Insert your frequency threshold as integer
    return df_freq_target[df_freq_target['tot'] > frequency_threshold]['Log-likelihood Ratio'].sort_values(ascending=False).iloc[range(50)]

In [ ]:
GetLexicalSpecificities(freq_term_DTM, logical_vector)

##### 1. Annotation and cleaning : ADD adjective and verbs as POS tag to keep

In [171]:
# 1. Annotation and cleaning : ADD adjective and verbs as POS tag to keep
cleaned_tweets = [CleaningText(sent, reduce = 'lemma', list_pos_to_keep = ['NOUN','ADJ','VERB'], Stopwords_to_add=['http']) for sent in list(df['Tweet Content'])]

In [172]:
# 2. Vectorisation
## 2.1 initialise with parameters : 
vectorized = CountVectorizer(lowercase = False, # Convert all characters to lowercase before tokenizing
                             min_df = 10, # Ignore terms that have a document frequency strictly lower than the given threshold 
                             max_df = 4500, # Ignore terms that have a document frequency strictly higher than the given threshold (corpus-specific stop words)
                             stop_words = stopwords.words('english'), # Remove the list of words provided
                             ngram_range = (1, 1), # Get the lower and upper boundary of the range of n-values for different word n-grams or char n-grams to be extracted
                             tokenizer=identity_tokenizer) # Override the string tokenization step while preserving the preprocessing and n-grams generation steps

#
freq_term_DTM = vectorized.fit_transform([[w for w, pos in sent] for sent in cleaned_tweets])

freq_term_DTM

/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:401: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ["'", 'b', 'c', 'e', 'f', 'g', 'h', 'j', 'l', 'n', 'p', 'r', 'u', 'v', 'w'] not in stop_words.
  % sorted(inconsistent)


<5000x1576 sparse matrix of type '<class 'numpy.int64'>'
	with 24770 stored elements in Compressed Sparse Row format>

In [175]:
logical_vector = df['compound_category'] == 'positive'

In [ ]:
GetLexicalSpecificities(freq_term_DTM, logical_vector)

In [178]:
df['Retweets received'].describe()

count      5000.000000
mean       6138.390800
std       20779.908625
min           0.000000
25%         148.000000
50%         733.000000
75%        3209.750000
max      459154.000000
Name: Retweets received, dtype: float64

In [180]:
bins = [-np.inf, 148, 733, 3209, 459154]
names = ['low', 'medium', 'high', 'very_high']
df['Retweets_received_category']  = pd.cut(df['Retweets received'], bins, labels=names, include_lowest =True)

In [185]:
logical_vector = df['Retweets_received_category'] == 'low'

In [186]:
GetLexicalSpecificities(freq_term_DTM, logical_vector)

uniform              30.618825
padmantrailer        30.104252
falta                30.069486
só                   30.033862
craquedaclaro        30.033862
carro                30.033862
ganhar               30.033862
concorra             30.033862
virar                30.033862
irresponsibletour    28.618825
agora                 6.743840
republic              5.365328
serve                 5.365328
wear                  5.223972
srbachchan            5.043400
jai                   5.043400
nightschool           4.917869
superhero             4.780366
remember              4.605279
wizkhalifa            4.488185
kevinhart4real        4.180904
gameofgames           4.043400
ho                    3.905897
writes                3.780366
site                  3.780366
democratic            3.195403
country               3.065095
jlo                   3.043400
kkwbeauty             2.917869
shop                  2.721472
creator               2.721472
lo                    2.458438
presiden